In [1]:
from twython import Twython as tw
from twython import TwythonError

In [2]:
# App keys and secrets stored as environment variables
import os
APP_KEY = os.environ.get('TWITTER_APP_KEY')
APP_SECRET = os.environ.get('TWITTER_APP_SECRET')

Access_Token = os.environ.get('TWITTER_ACCESS_TOKEN')
Access_Secret = os.environ.get('TWITTER_ACCESS_SECRET')

twitter = tw(APP_KEY, APP_SECRET, Access_Token, Access_Secret)

In [3]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [4]:
# Define a number of queries

query_terms = {'bitcoin', 'altcoin', 'cryptocurrency', 'ethereum'}

# Then loop through queries
q_collection = {}

for x in query_terms:
    q_collection[x] = {'q': x,  
        'result_type': 'popular',
        'count': 100,
        'lang': 'en',
        }

In [5]:
# Search those queries on Twitter and create a dictionary of responses
dict_ = {'user': [], 'date': [], 'text': [], 'favorite_count': [], 'user_ID': []}  
for a in query_terms:
    for status in twitter.search(**q_collection[a])['statuses']:  
        dict_['user'].append(status['user']['screen_name'])
        dict_['date'].append(status['created_at'])
        dict_['text'].append(status['text'])
        dict_['favorite_count'].append(status['favorite_count'])
        dict_['user_ID'].append(status['user'])
        
# Make into a dataframe
df = pd.DataFrame(dict_)  


In [6]:
# Write dict_ to tweets.db. 
# Basically, want to save the data to a database before messing with it, so can go back at a later date.

import sqlite3

connection = sqlite3.connect('tweets.db')
cursor = connection.cursor()

query = "INSERT INTO tweets VALUES (NULL,?,?,?,?)"

df_clean = df.drop('user_ID', axis = 1)
data_for_query = [tuple(x) for x in df_clean.values]
cursor.executemany(query, data_for_query)
connection.commit()

connection.close()

In [7]:
# Remove duplicates to find individual users 
# Then pull user names & ID's

df = df.drop_duplicates(subset='user', keep='first', inplace=False)
influencers = df.loc[:, 'user']



In [8]:
# Store the newly found users in a Twitter list. 

my_user_id = 705469309
my_list_name = 'tweet_smart_list' #list name - create this on Twitter
# Create dictionary of my lists
my_lists_dict = twitter.show_owned_lists(user_id = my_user_id)  

In [9]:
#Split so that running code doesn't keep sending requests to Twitter.

my_lists_df = pd.DataFrame(my_lists_dict['lists']) 
names = my_lists_df.loc[my_lists_df.name == my_list_name]
my_list_id = int(names.id)
list_slug = names.slug

# Stores users in twitter list, printing any errors for debugging
for user in influencers:
    try:
        twitter.add_list_member(list_id = my_list_id, screen_name=user)
    except TwythonError as e:
        print(e)